In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from repltilian import SwiftREPL
repl = SwiftREPL()

print("REPL is running !")
REPL is running !
 23>  


In [3]:
repl.add_reload_file("demo.swift")

In [4]:
repl.run("""
var point = Point<Float>(x: 1, y: 2)
""", reload=True);

 58.  
 59. var point = Point<Float>(x: 1, y: 2) 
 60.  
point: Point<Float> = {
  x = 1
  y = 2
}
 60>  


In [5]:
repl.run("""//cpp
var point1 = Point<Float>(x: 5, y: 2)
var point2 = Point<Float>(x: 5, y: 2)
let result = point1 + point2 + point
result.translate(dx: 1, dy: 100)
""");

 60> //cpp 
 61. var point1 = Point<Float>(x: 5, y: 2) 
 62. var point2 = Point<Float>(x: 5, y: 2) 
 63. let result = point1 + point2 + point 
 64. result.translate(dx: 1, dy: 100) 
 65.  
$R0: Point<Float> = {
  x = 12
  y = 106
}
point1: Point<Float> = {
  x = 5
  y = 2
}
point2: Point<Float> = {
  x = 5
  y = 2
}
result: Point<Float> = {
  x = 11
  y = 6
}
 65>  


In [6]:
repl.vars["point2"].json()

{'x': 5, 'y': 2}

In [7]:
repl.vars["result"].dtype, repl.vars["result"].json()

('Point<Float>', {'x': 11, 'y': 6})

In [8]:
repl.vars["result"] = {'x': -1.5, 'y': -1.2}

In [9]:
repl.vars["result"].json()

{'x': -1.5, 'y': -1.2}

In [10]:
repl.vars.create("y", "Array<Point<Float>>", [{'x': -1.5, 'y': -1.2}]*10)

In [11]:
repl.run("result.translate(dx: 1, dy: 100)");

 72> result.translate(dx: 1, dy: 100)
$R1: Point<Float> = {
  x = -0.5
  y = 98.800003
}
 73>  


In [13]:
repl.vars["y"] = [{'x': -1, 'y': -1}] * 5
repl.vars["y"].json()

[{'x': -1, 'y': -1},
 {'x': -1, 'y': -1},
 {'x': -1, 'y': -1},
 {'x': -1, 'y': -1},
 {'x': -1, 'y': -1}]

In [14]:
repl.run("""//cpp
y[0]
""");

 79> //cpp 
 80. y[0] 
 81.  
$R2: Point<Float> = {
  x = -1
  y = -1
}
 81>  
